In [ ]:
import json
import nltk
import os
import pymorphy2
import nltk
import re
import numpy as np
import gensim
import spacy

import gensim.downloader
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from nltk.corpus import stopwords

from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support

from sklearn.linear_model import SGDClassifier

In [ ]:
nltk.download(
    'stopwords',
    download_dir=os.getcwd(),
)
nltk.data.path.append(os.getcwd())

stop_words = set(stopwords.words('russian'))
stop_words.add('свой')

In [ ]:
nlp = spacy.load('ru_core_news_lg')

In [ ]:
regex = '[^A-Za-z\sаАбБвВгГдДеЕёЁжЖзЗиИйЙкКлЛмМнНоОпПрРсСтТуУфФхХцЦчЧшШщЩъЪыЫьЬэЭюЮяЯ]'
regexRussian = '[^\sаАбБвВгГдДеЕёЁжЖзЗиИйЙкКлЛмМнНоОпПрРсСтТуУфФхХцЦчЧшШщЩъЪыЫьЬэЭюЮяЯ]'
rgRu = re.compile(regexRussian)
rg = re.compile(regex)
morph = pymorphy2.MorphAnalyzer()

In [ ]:
pos_convert_dict = {
    'ADVB': 'ADV',
    'ADJF': 'ADJ',
    'NPRO': 'NOUN'
}

In [ ]:
def tokenization_condition(token):
    return all([
        token.ent_type == 0,
        rgRu.match(token.text) is None,
        token.is_alpha,
        not token.like_num,
        not token.like_email,
    ])

def tokenize_spacy(text: str):
    doc = nlp(text)
    tokens = []
    for token in doc:
        if tokenization_condition(token):
            token = morph.parse(token.text)[0].normal_form
            tokens.append(token)
    return tokens

def tokenize_gensim(text: str):
    doc = nlp(text)
    tokens = []
    for token in doc:
        if tokenization_condition(token):
            tag = morph.parse(token.text)[0]
            token = tag.normal_form
            tokens.append(token)
    return tokens

def tokenize_gensim_pretrained(text: str):
    doc = nlp(text)
    tokens = []
    for token in doc:
        if tokenization_condition(token):
            tag = morph.parse(token.text)[0]
            token = tag.normal_form
            pos = tag.tag.POS
            if pos_convert_dict.get(pos) is not None:
                pos = pos_convert_dict[pos]
            if pos is not None:
                token = '_'.join([token, pos])
            tokens.append(token)
    return tokens

def process_fn(text: str):
    doc = nlp(text)
    text = re.sub(regex, "", text).lower()
    new_text = []
    for word in text.split(' '):
        if word not in stop_words:
            word = morph.parse(word)[0].normal_form
            new_text.append(word)
    text = ' '.join(new_text)
    return text

def process_fn_gensim(text: str):
    text = ' '.join(re.sub(regexRussian, "", text).lower().split())
    new_text = []
    for word in text.split(' '):
        if word not in stop_words:
            tag = morph.parse(word)[0]
            word = tag.normal_form
            pos = tag.tag.POS
            if pos is None:
                continue
            if pos == 'ADVB':
                pos = 'ADV'
            if pos == 'ADJF':
                pos = 'ADJ'
            word = '_'.join([word, pos])
            new_text.append(word)
    text = ' '.join(new_text)
    return text

In [ ]:
class TextData:
    def __init__(self, filepath, process_fn = process_fn):
        with open(filepath) as file:
            data = json.loads(file.read())
        self.__labels = np.array([])
        self.__raw_text = []
        for d in data['data']:
            self.__labels = np.append(self.__labels, d['evaluation'])
            self.__raw_text.append(d['speech'])
        
    @property
    def labels(self):
        return self.__labels
    
    @property
    def text(self):
        return self.__raw_text
    
    @property
    def text_and_labels(self):
        return list(zip(self.text, self.labels))

In [ ]:
class EmbVectorizer:
    #ruscorpora_model = gensim.downloader.load('word2vec-ruscorpora-300')
    def __init__(self, preprocessor = None, tokenizer = None, stop_words = None):
        self.__preprocessor = preprocessor
        if preprocessor is None:
            self.__preprocessor = lambda x : x
            
        self.__tokenizer = tokenizer
        if tokenizer is None:
            self.__tokenizer = lambda x : x.split()
            
        self.__stop_words = stop_words
        if stop_words is None:
            self.__stop_words = {}
            
    def __process(self, raw_documents, y = None):
        raw_documents = list(map(self.__preprocessor, raw_documents))
        raw_documents = list(map(lambda x: ' '.join(list(filter(lambda x: x not in self.__stop_words, x.split()))), raw_documents))
        tokens = list(map(self.__tokenizer, raw_documents))
        return tokens
        
    def fit(self, raw_documents, y = None):
        self.__fit(raw_documents)
        return self
        
    def transform(self, raw_documents):
        return self.__transform(raw_documents)
    
    def __fit(self, raw_documents):
        tokens = self.__process(raw_documents)
        docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokens)]
        self.__model = Doc2Vec(docs, vector_size=300, window=5, min_count=0, workers=4)
        
    def __transform(self, tokens: list):
        vec = []
        for token in self.__process(tokens):
            vec.append(self.__model.infer_vector(token))
        return np.array(vec)
    
    def __get_feature(self, tokens: list):
        vector_bag = []
        for t in tokens:
            if EmbVectorizer.ruscorpora_model.has_index_for(t):
                vector_bag.append(EmbVectorizer.ruscorpora_model.get_vector(t))
            else:
                print(t)
        return np.sum(vector_bag, axis=0) / len(vector_bag)
    
    def features(self):
        return self.__features

In [ ]:
class PretrainedVectorizer:
    ruscorpora_model = gensim.downloader.load('word2vec-ruscorpora-300')
    def __init__(self, preprocessor = None, tokenizer = None, stop_words = None):
        self.__preprocessor = preprocessor
        if preprocessor is None:
            self.__preprocessor = lambda x : x
            
        self.__tokenizer = tokenizer
        if tokenizer is None:
            self.__tokenizer = lambda x : x.split()
            
        self.__stop_words = stop_words
        if stop_words is None:
            self.__stop_words = {}
            
    def fit(self, raw_documents, y = None):
        return self
    
    def transform(self, raw_documents):
        return self.__transform(self.__process(raw_documents))
    
    def __process(self, raw_documents, y = None):
        raw_documents = list(map(self.__preprocessor, raw_documents))
        raw_documents = list(map(lambda x: ' '.join(list(filter(lambda x: x not in self.__stop_words, x.split()))), raw_documents))
        return list(map(self.__tokenizer, raw_documents))
          
    def __transform(self, tokens: list):
        vector_bag = []
        for tt in tokens:
            doc_vector = []
            for t in tt:
                if PretrainedVectorizer.ruscorpora_model.has_index_for(t):
                    doc_vector.append(PretrainedVectorizer.ruscorpora_model.get_vector(t))
            if len(doc_vector) == 0:
                vector_bag.append(list(np.zeros((300))))
            else:
                vector_bag.append(list(np.sum(doc_vector, axis=0) / len(doc_vector))) 
        return np.array(vector_bag)

In [ ]:
class Model:
    def __init__(self, pipeline = None, params = {}):
        self.__clf = GridSearchCV(
            estimator=pipeline, 
            param_grid=params,
            n_jobs=-1, verbose=1,
        )
        
    def fit(self, raw_documents, y = None):
        self.__clf.fit(raw_documents, y)
        
    def precision_recall_f1_support(self, raw_documents, y = None):
        return precision_recall_fscore_support(
            y, 
            self.__clf.predict(raw_documents),
            average='weighted',
        )

In [ ]:
train_data = TextData('data/new_eval_train.json')
test_data = TextData('data/new_eval_test.json')

In [ ]:
steps_binary = [
    ('vect', CountVectorizer(
        tokenizer=tokenize_spacy,
        binary=True,
        #stop_words=list(stop_words),               
    ))
]

steps_count = [
    ('vect', CountVectorizer(
        tokenizer=tokenize_spacy,
        #stop_words=list(stop_words),
    ))
]

steps_tfidf = [
    ('vect', CountVectorizer(tokenizer=tokenize_spacy)),
    ('tfidf', TfidfTransformer())
]

steps_emb = [
    ('emb', EmbVectorizer(
        tokenizer=tokenize_gensim, 
        stop_words=stop_words
    )),
]

steps_pretrained = [
    ('emb', PretrainedVectorizer(
        tokenizer=tokenize_gensim_pretrained, 
        stop_words=stop_words
    ))
]

best_parameters_sgd =  {
    # params for clf
    'clf__alpha': (0.0001,),
    'clf__average': (True,),
    'clf__eta0': (0.01,),
    'clf__learning_rate': ('optimal',),
    'clf__max_iter': (200, 250, 275, 1000,),
    'clf__penalty': ('l2',),
}

optimization_parameters_sgd_lg = {
    # params for clf
    'clf__loss': ('hinge', 'modified_huber', 'squared_hinge' , 'log'),
    'clf__learning_rate': ('optimal', 'invscaling', 'adaptive'),
    'clf__alpha': (0.0001, 0.00001),
    'clf__epsilon': (0.1, 0.01, 0.001),
    'clf__penalty': ('l2', 'l1', 'elasticnet'),
    'clf__max_iter': (275,1000,1500,2000),
    'clf__average': (True, False),
    'clf__eta0': (0.01, 0.001, 0.0001)
}

optimization_parameters_sgd_sm = {
    # params for clf
    'clf__loss': ( 'squared_hinge' ,),
    'clf__learning_rate': ('optimal', 'adaptive'),
    'clf__alpha': (0.0001, 0.00001),
    'clf__epsilon': (0.1, 0.01),
    'clf__penalty': ('l2', 'l1', 'elasticnet'),
    'clf__max_iter': (250, 275, 1000),
    'clf__average': (True, False),
    'clf__eta0': (0.01, 0.001, 0.0001)
}

In [135]:
# Stochastic Gradient Descent (pretrained emb)
steps_sgd = steps_pretrained.copy()
steps_sgd.append(('clf', SGDClassifier()))
    
sgd_emb_model = Model(
    pipeline = Pipeline(steps_sgd),
    params = best_parameters_sgd,
)

sgd_emb_model.fit(train_data.text, train_data.labels)
sgd_emb_model.precision_recall_f1_support(test_data.text, test_data.labels)

/opt/homebrew/Caskroom/miniforge/base/envs/conda_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


(0.6493056416446652, 0.6529630439536409, 0.6275007707304155, None)

In [ ]:
# Stochastic Gradient Descent (pretrained emb)
steps_sgd = steps_emb.copy()
steps_sgd.append(('clf', SGDClassifier()))
    
sgd_emb_model = Model(
    pipeline = Pipeline(steps_sgd),
    params = best_parameters_sgd,
)

sgd_emb_model.fit(train_data.text, train_data.labels)
sgd_emb_model.precision_recall_f1_support(test_data.text, test_data.labels)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/opt/homebrew/Caskroom/miniforge/base/envs/conda_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


In [ ]:
# Stochastic Gradient Descent (binary)
steps_sgd = steps_binary.copy()
steps_sgd.append(('clf', SGDClassifier()))
pipeline_sgd = Pipeline(steps_sgd)

parameters_sgd = best_parameters_sgd 

clf_sgd = GridSearchCV(
    estimator=pipeline_sgd, 
    param_grid=parameters_sgd,
    n_jobs=-1, verbose=1,
)

clf_sgd.fit(train_data.text, train_data.labels)
precision_recall_fscore_support(
    test_data.labels, 
    clf_sgd.predict(test_data.text),
    average='weighted',
)

In [ ]:
# Stochastic Gradient Descent (count)
steps_sgd = steps_count.copy()
steps_sgd.append(('clf', SGDClassifier()))
pipeline_sgd = Pipeline(steps_sgd)

parameters_sgd = best_parameters_sgd

clf_sgd = GridSearchCV(
    estimator=pipeline_sgd, 
    param_grid=parameters_sgd,
    n_jobs=-1, verbose=1,
)

clf_sgd.fit(train_data.text, train_data.labels)
precision_recall_fscore_support(
    test_data.labels, 
    clf_sgd.predict(test_data.text),
    average='weighted',
)

clf_sgd.score(train_data.text, train_data.labels)

In [ ]:
# Stochastic Gradient Descent (tfidf)
steps_sgd = steps_tfidf.copy()
steps_sgd.append(('clf', SGDClassifier()))
pipeline_sgd = Pipeline(steps_sgd)

parameters = best_parameters_sgd

clf_sgd = GridSearchCV(
    estimator=pipeline_sgd, 
    param_grid=parameters,
    n_jobs=-1, verbose=1,
)

clf_sgd.fit(train_data.text, train_data.labels)
precision_recall_fscore_support(
    test_data.labels, 
    clf_sgd.predict(test_data.text),
    average='weighted',
)